In [2]:
#Instalar una biblioteca que utilizaremos más adelante para corregir de manera automática los números que se han escrito como texto y no como número (si no se tenía instalada previamente)

#!pip install word2number

In [3]:
import pandas as pd
import numpy as np
import re
from word2number import w2n

In [4]:
#Configurar que pandas muestre todas las columnas y todas las filas del dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
#Descargar el csv

df = pd.read_csv('1_HR_RAW_DATA.csv', index_col = 0)

### EDA BÁSICO PARA ENTENDER NUESTROS DATOS

In [7]:
#Hacer la función para sacar el EDA

def eda_basico(df):

    print('🌷Ejemplo de datos del DF:')
    display(df.head(3))
    display(df.tail(3))
    display(df.sample(3))
    print('________________________________________________________________________________________________________')

    print('🌻Número de Filas:')
    display(df.shape[0])
    print('________________________________________________________________________________________________________')

    print('🌱Número de Columnas:')
    display(df.shape[1])
    print('________________________________________________________________________________________________________')

    print('🌼Información de la tabla:')
    display(df.info())
    print('________________________________________________________________________________________________________')

    print('🌑Nombre de las columnas:')
    display(df.columns)
    print('________________________________________________________________________________________________________')

    print('🍄Descripción de los datos numéricos:')
    display(df.describe().T)
    print('________________________________________________________________________________________________________')

    print('🌋Descripción de los datos no-numéricos:')
    display(df.describe(include='object').T)
    print('________________________________________________________________________________________________________')

    print('🍂Saber si hay datos únicos:')
    display(df.nunique())
    print('________________________________________________________________________________________________________')

    print('🐖Que datos son nulos por columnas:')
    display(df.isnull().sum())
    print('________________________________________________________________________________________________________')

    print('🐲Filas duplicadas:')
    total_duplicados = df.duplicated().sum()
    if total_duplicados > 0:
        print(f'cantidad de duplicados: {total_duplicados}')
        print('Primeros duplicados')
        display(df[df.duplicated()].head(3))
    else:
        print('No hay duplicados')
    print('________________________________________________________________________________________________________')

In [ ]:
eda_basico(df)

Revisamos el nombre de las columnas

In [8]:
df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'employeecount',
       'employeenumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NUMCOMPANIESWORKED',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TOTALWORKINGYEARS', 'TrainingTimesLastYear', 'WORKLIFEBALANCE',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YEARSWITHCURRMANAGER', 'SameAsMonthlyIncome', 'DateBirth', 'Salary',
       'RoleDepartament', 'NUMBERCHILDREN', 'RemoteWork'],
      dtype='object')

In [8]:
#Crear una función para renombrar las columnas de manera uniforme

def transformar(nombre):
        # Si el nombre está completamente en mayúsculas, se pone en minúsculas
        if nombre.isupper():
            return nombre.lower()
        # Si hay una mayúscula para indicar una segunda o tercera palabra, se separa por esa mayúsculas y convierte la palabra a minúsculas
        else:
            nombre = re.sub(r'(?<!^)(?=[A-Z])', '_', nombre)
            return nombre.lower()

# Se aplica la función a cada columna
df.columns = [transformar(i) for i in df.columns]
df.columns

Index(['age', 'attrition', 'business_travel', 'daily_rate', 'department',
       'distance_from_home', 'education', 'education_field', 'employeecount',
       'employeenumber', 'environment_satisfaction', 'gender', 'hourly_rate',
       'job_involvement', 'job_level', 'job_role', 'job_satisfaction',
       'marital_status', 'monthly_income', 'monthly_rate',
       'numcompaniesworked', 'over18', 'over_time', 'percent_salary_hike',
       'performance_rating', 'relationship_satisfaction', 'standard_hours',
       'stock_option_level', 'totalworkingyears', 'training_times_last_year',
       'worklifebalance', 'years_at_company', 'years_in_current_role',
       'years_since_last_promotion', 'yearswithcurrmanager',
       'same_as_monthly_income', 'date_birth', 'salary', 'role_departament',
       'numberchildren', 'remote_work'],
      dtype='object')

In [9]:
#Se renombra manualmente las columnas que no tenían indicadores para entender la separación de palabras

df.rename(columns={
    "employeecount": "employee_count", 
    "employeenumber":"employee_number", 
    "numcompaniesworked":"num_companies_worked", 
    "over18":"over_18", 
    "totalworkingyears":"total_working_years", 
    "worklifebalance":"worklife_balance", 
    "yearswithcurrmanager":"years_with_curr_manager", 
    "numberchildren":"number_children"
    }, inplace=True)
df.columns

Index(['age', 'attrition', 'business_travel', 'daily_rate', 'department',
       'distance_from_home', 'education', 'education_field', 'employee_count',
       'employee_number', 'environment_satisfaction', 'gender', 'hourly_rate',
       'job_involvement', 'job_level', 'job_role', 'job_satisfaction',
       'marital_status', 'monthly_income', 'monthly_rate',
       'num_companies_worked', 'over_18', 'over_time', 'percent_salary_hike',
       'performance_rating', 'relationship_satisfaction', 'standard_hours',
       'stock_option_level', 'total_working_years', 'training_times_last_year',
       'worklife_balance', 'years_at_company', 'years_in_current_role',
       'years_since_last_promotion', 'years_with_curr_manager',
       'same_as_monthly_income', 'date_birth', 'salary', 'role_departament',
       'number_children', 'remote_work'],
      dtype='object')

# Limpieza de datos

In [ ]:
#Revisar en general los valores de las columnas para posteriormente corregir los errores


def unicos(df): 
    for i in df:
        print(f"\n --------------------------------------------------------------------------- \n {i}")
        print(f"Nulos: {df[i].isnull().sum()}")
        print(f"Valores distintos: {df[i].nunique()}")
        print(f"Tipo de dato: {df[i].dtype}")
        print("\nValores distintos y cuántas veces se repiten:")
        display(df[i].value_counts())

#Aplicar la función para revisar los valores únicos de cada columna
unicos(df)


In [11]:
# Revisar los valores de las columnas, una por una

def unicos_individual(x): 
    print(f"\n --------------------------------------------------------------------------- \n {x}")
    print(f"Nulos: {df[x].isnull().sum()}")
    print(f"Valores distintos: {df[x].nunique()}")
    print(f"Tipo de dato: {df[x].dtype}")
    print("\nValores distintos y cuántas veces se repiten:")
    display(df[x].value_counts())        

### Columna age

In [ ]:
# Crear una función, utilizando la biblioteca word2number, para convertir los números escritos como texto a números enteros

# miramos cuántos valores distintos tenemos antes de la modificación
print(unicos_individual("age"))

#Crear una función, utilizando la biblioteca word2number, para convertir los números escritos como texto a números enteros
def convertir_palabras_a_numero(x):
    try:
        return w2n.word_to_num(x)
    except:
        return x  # Devuelve el valor original si no puede convertir

#Aplicar la función a la columna 'age'
df["age"] = df["age"].apply(convertir_palabras_a_numero)

#Comprobar los cuántos valores distintos tenemos tras la modificación para saber si se ha realizado correctamente
print(unicos_individual("age"))

### Columna attrition / education_field

In [ ]:
# Convertir "attrition" y "education_field" a minúsculas y quitar espacios
df[["attrition", "education_field"]] = df[["attrition", "education_field"]].apply(lambda col: col.str.lower())

print(unicos_individual("attrition"))
print(unicos_individual("education_field"))

### Columna business_travel

In [ ]:
# business_travel tiene valores únicos correctos pero muchos nulos. La cliente nos ha confirmado que puede asumirse que quien no ha contestado no ha viajado (correspondería al valor non-travel)

print(unicos_individual("business_travel"))
df["business_travel"] = df["business_travel"].fillna("non-travel")
print(unicos_individual("business_travel"))

### Columna distance_from_home

In [ ]:
# distance_from_home: originalmente tiene valores negativos. La cliente nos confirma que debemos asumir su valor absoluto.

print(unicos_individual("distance_from_home"))
df["distance_from_home"] = df["distance_from_home"].abs()

print(unicos_individual("distance_from_home"))


### Columna gender

In [ ]:
# gender: la cliente nos indica que el valor "0" será "M" y el valor "1" será "F"

print(unicos_individual("gender"))
df["gender"] = df["gender"].replace({0: "M", 1: "F"})

print(unicos_individual("gender"))


### Columna job_role

In [ ]:
# job_role: homogeneizamos los valores a minúsculas

print(unicos_individual("job_role"))
df["job_role"] = df["job_role"].str.lower()

print(unicos_individual("job_role"))

### Columna marital_status

In [ ]:
# marital_status: homogeneizamos los valores a minúsculas y corregimos "marreid" por "married"

print(unicos_individual("marital_status"))
df["marital_status"] = df["marital_status"].str.lower()
df["marital_status"] = df["marital_status"].str.replace("marreid", "married")

print(unicos_individual("marital_status"))



### Columna monthly_income

In [ ]:
# monthly_income: renombramos columna para indicar las unidades monetarias confirmadas por la cliente
print(unicos_individual("monthly_income"))
df = df.rename(columns={"monthly_income": "monthly_income_$"})

print(unicos_individual("monthly_income"))


### Columna standard_hours

In [ ]:
# standard_hours: la cliente confirma que los nulos son full-time. Cambiamos "80,0" a "part-time" y los nulos a "full-time".

print(unicos_individual("standard_hours"))
df["standard_hours"] = df["standard_hours"].str.replace("80,0", "part-time")
df["standard_hours"] = df["standard_hours"].fillna("full-time")

print(unicos_individual("standard_hours"))


### Columna remote_work

In [ ]:
# remote_work: sólo debería haber "yes" y "no" por lo que modificamos los datos aportados.

print(unicos_individual("remote_work"))
df["remote_work"] = df["remote_work"].str.lower().replace({"1": "yes", "true": "yes", "0": "no", "false": "no"})

print(unicos_individual("remote_work"))


## Eliminar columnas

In [ ]:
#Eliminamos las columnas que no tienen información que se pueda usar para el análisis
columnas_a_eliminar = ["daily_rate", "department", "employee_count", "employee_number", "hourly_rate", "monthly_rate", "over_18", "training_times_last_year", "years_in_current_role", "same_as_monthly_income", "date_birth", "salary", "role_departament", "number_children"]
columnas_existentes = df.columns
print(columnas_a_eliminar)
print(columnas_existentes)
for col in columnas_a_eliminar:
    if col in columnas_existentes:
        print(f"Eliminando columna: {col}")
        df = df.drop(columns=[col], axis=1)
    else:
        print(f"La columna {col} no existe en el DataFrame y no se eliminará.")
print(df.columns)

In [ ]:
def eda_basico(df):

    print('🌷Ejemplo de datos del DF:')
    display(df.head(3))
    display(df.tail(3))
    display(df.sample(10))
    print('________________________________________________________________________________________________________')

    print('🌻Número de Filas:')
    display(df.shape[0])
    print('________________________________________________________________________________________________________')

    print('🌱Número de Columnas:')
    display(df.shape[1])
    print('________________________________________________________________________________________________________')

    print('🌼Información de la tabla:')
    display(df.info())
    print('________________________________________________________________________________________________________')

    print('🌑Nombre de las columnas:')
    display(df.columns)
    print('________________________________________________________________________________________________________')

    print('🍄Descripción de los datos numéricos:')
    display(df.describe().T)
    print('________________________________________________________________________________________________________')

    print('🌋Descripción de los datos no-numéricos:')
    display(df.describe(include='object').T)
    print('________________________________________________________________________________________________________')

    print('🍂Saber si hay datos únicos:')
    display(df.nunique())
    print('________________________________________________________________________________________________________')

    print('🐖Que datos son nulos por columnas:')
    display(df.isnull().sum())
    print('________________________________________________________________________________________________________')

    print('🐲Filas duplicadas:')
    total_duplicados = df.duplicated().sum()
    if total_duplicados > 0:
        print(f'cantidad de duplicados: {total_duplicados}')
        print('Primeros duplicados')
        display(df[df.duplicated()].head(3))
    else:
        print('No hay duplicados')
    print('________________________________________________________________________________________________________')


In [ ]:
eda_basico(df)

### Cambiamos '.', por ',' para después poder pasar la columna a INT

In [27]:
columnas_float = ["monthly_income_$", "performance_rating", "total_working_years", "worklife_balance"]

# Convertir a string y reemplazar punto por coma
for col in columnas_float:
    df[col] = df[col].str.replace(',', '.')

# Comprobación (opcional): mostrar un ejemplo de los primeros valores
print(df[columnas_float].head())

  monthly_income_$ performance_rating total_working_years worklife_balance
0          19537.0                3.0                 NaN              3.0
1          19999.0                3.0                34.0              3.0
2          19232.0                3.0                22.0              NaN
3          17169.0                3.0                 NaN              NaN
4              NaN                3.0                 NaN              3.0


In [28]:
df.sample(10)

,age,attrition,business_travel,distance_from_home,education,education_field,environment_satisfaction,gender,job_involvement,job_level,job_role,job_satisfaction,marital_status,monthly_income_$,num_companies_worked,over_time,percent_salary_hike,performance_rating,relationship_satisfaction,standard_hours,stock_option_level,total_working_years,worklife_balance,years_at_company,years_since_last_promotion,years_with_curr_manager,remote_work
43,36,yes,travel_rarely,16,4,NaN,3,F,4,1,laboratory technician,1,single,NaN,1,NaN,16,3.0,3,part-time,0,18.0,3.0,17,15,14,yes
891,24,no,non-travel,10,4,marketing,4,F,3,2,sales executive,3,divorced,NaN,1,Yes,12,3.0,4,full-time,1,NaN,4.0,5,0,3,yes
557,47,yes,non-travel,9,3,NaN,3,M,1,4,sales executive,3,married,12936.0,7,No,11,NaN,3,full-time,0,25.0,1.0,23,14,10,yes
1006,37,no,travel_rarely,25,2,NaN,3,F,2,2,healthcare representative,4,NaN,5731.0,7,No,13,3.0,3,full-time,2,9.0,3.0,6,1,3,yes
1212,32,no,non-travel,5,2,life sciences,4,F,2,2,research scientist,2,married,5878.0,3,No,12,3.0,1,full-time,1,12.0,3.0,7,2,5,yes
757,28,no,non-travel,27,3,NaN,2,F,1,2,manufacturing director,1,divorced,4877.0,0,NaN,21,4.0,2,full-time,1,6.0,2.0,5,0,0,yes
618,33,no,non-travel,2,3,human resources,2,M,3,1,human resources,3,NaN,NaN,1,No,13,3.0,4,full-time,1,5.0,3.0,5,1,4,yes
512,44,no,non-travel,28,3,life sciences,4,M,3,4,research director,1,married,NaN,3,No,13,3.0,3,full-time,1,NaN,4.0,20,14,17,yes
308,33,no,non-travel,10,3,NaN,2,M,2,2,sales executive,4,NaN,4682.0,3,No,14,3.0,3,full-time,0,NaN,2.0,7,0,1,no
1435,38,no,travel_rarely,18,3,medical,2,M,1,2,healthcare representative,4,married,NaN,3,NaN,16,3.0,3,full-time,1,NaN,3.0,1,1,0,yes


In [33]:
df.to_csv('2_HR_DATA_clean.csv')